In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_white"

# Load the dataset
stocks_data = pd.read_csv("stocks.csv")

# Display the first few rows of the dataset
print(stocks_data.head())

  Ticker        Date        Open        High         Low       Close  \
0   AAPL  2023-02-07  150.639999  155.229996  150.639999  154.649994   
1   AAPL  2023-02-08  153.880005  154.580002  151.169998  151.919998   
2   AAPL  2023-02-09  153.779999  154.330002  150.419998  150.869995   
3   AAPL  2023-02-10  149.460007  151.339996  149.220001  151.009995   
4   AAPL  2023-02-13  150.949997  154.259995  150.919998  153.850006   

    Adj Close    Volume  
0  154.414230  83322600  
1  151.688400  64120100  
2  150.639999  56007100  
3  151.009995  57450700  
4  153.850006  62199000  


##Adj Close: The adjusted closing price, which accounts for all corporate actions such as dividends, stock splits, etc.
##Volume: The number of shares traded during the day.

#Descriptive Statistics

##Descriptive Statistics will provide summary statistics for each stock in the dataset. We’ll look at measures such as mean, median, standard deviation, and more for the Close prices:

In [ ]:
descriptive_stats = stocks_data.groupby('Ticker')['Close'].describe()
print(descriptive_stats)

        count        mean        std         min         25%         50%  \
Ticker                                                                     
AAPL     62.0  158.240645   7.360485  145.309998  152.077499  158.055000   
GOOG     62.0  100.631532   6.279464   89.349998   94.702501  102.759998   
MSFT     62.0  275.039839  17.676231  246.270004  258.742500  275.810013   
NFLX     62.0  327.614677  18.554419  292.760010  315.672493  325.600006   

               75%         max  
Ticker                          
AAPL    165.162506  173.570007  
GOOG    105.962503  109.459999  
MSFT    287.217506  310.649994  
NFLX    338.899994  366.829987  


##Count:The number of observations or trading days included in the dataset
##Mean:The average closing price
##Standard Deviation:Measures the amount of variation or dispersion of closing prices
##Minimum:The lowest closing price in the dataset
##25th Percentile:25% of the closing prices are below this value
##Median (50%):The middle value of the closing prices
##75th Percentile:75% of the closing prices are below this value
##Maximum:The highest closing price in the dataset

---
##More the standard deviation greater the price variability(fluctuations)

In [ ]:
stocks_data['Data']=pd.to_datetime(stocks_data['Date'])

In [ ]:
pivot_data=stocks_data.pivot(index='Data',columns='Ticker',values='Close')

In [ ]:
fig=make_subplots(rows=1,cols=1)

In [ ]:
for columns in pivot_data.columns:
  fig.add_trace(
      go.Scatter(x=pivot_data.index,y=pivot_data[columns],name=columns),
      row=1,col=1
      )


In [ ]:
fig.update_layout(
    title_text='Stock Price Over Time',
    xaxis_title='Date',
    yaxis_title='Stock Price',
    legend_title='Ticker',
    showlegend=True
)

fig.show()

**Trend:** Each stock shows its unique trend over time. For instance, AAPL and MSFT exhibit a general upward trend in this period.
---
**Volatility:** There is noticeable volatility in the stock prices. For example, NFLX shows more pronounced fluctuations compared to others.
---
**Comparative Performance:** When comparing the stocks, MSFT and NFLX generally trade at higher price levels than AAPL and GOOG in this dataset.
---

#Volatility Analysis

In [ ]:
volatility = pivot_data.std().sort_values(ascending=False)

fig = px.bar(volatility,
             x=volatility.index,
             y=volatility.values,
             labels={'y': 'Standard Deviation', 'x': 'Ticker'},
             title='Volatility of Closing Prices (Standard Deviation)')

# Show the figure
fig.show()

#Correlation Analysis

In [ ]:
correlation_matrix = pivot_data.corr()

fig = go.Figure(
    data=go.Heatmap(
    x=pivot_data.columns,
    y=pivot_data.columns,
    z=correlation_matrix.values,
    colorscale='RdBu',
    colorbar=dict(title='Correlation')
))

fig.update_layout(
    title='Correlation Analysis of Closing prices',
    xaxis_title='Ticker',
    yaxis_title='Ticker',
    showlegend=False
)

fig.show()

##From the heatmap, we can observe that there are varying degrees of positive correlations between the stock prices, with some pairs showing stronger correlations than others. For instance, AAPL and MSFT seem to have a relatively higher positive correlation.

In [ ]:
percentage_change = ((pivot_data.iloc[-1] - pivot_data.iloc[0]) / pivot_data.iloc[0]) * 100

fig = px.bar(percentage_change,
             x=percentage_change.index,
             y=percentage_change.values,
             labels={'y': 'Percentage Change (%)', 'x': 'Ticker'},
             title='Percentage Change in Closing Prices')

# Show the plot
fig.show()

MSFT: The highest positive change of approximately 16.10%.
---
AAPL: Exhibited a positive change of approximately 12.23%. It indicates a solid performance, though slightly lower than MSFT’s.
---
GOOG: Showed a slight negative change of about -1.69%. It indicates a minor decline in its stock price over the observed period.
---
NFLX: Experienced the most significant negative change, at approximately -11.07%. It suggests a notable decrease in its stock price during the period.
---

#Comparitive Analysis

In [ ]:
daily_returns = pivot_data.pct_change().dropna()


###pivot_data.pct_change() calculates the daily percentage change for each stock's closing prices. This gives the daily returns for each stock.

In [ ]:
avg_daily_return = daily_returns.mean()
risk = daily_returns.std()

###avg_daily_return = daily_returns.mean(): Computes the average daily return for each stock. This gives the mean of the daily percentage changes for each stock.

###risk = daily_returns.std(): Calculates the standard deviation of daily returns, which is used as a measure of risk (the higher the standard deviation, the higher the volatility/risk).

In [ ]:
# Creating a DataFrame for plotting
risk_return_df = pd.DataFrame({'Risk': risk, 'Average Daily Return': avg_daily_return})

fig = go.Figure()

In [ ]:
# Add scatter plot points
fig.add_trace(go.Scatter(
    x=risk_return_df['Risk'],
    y=risk_return_df['Average Daily Return'],
    mode='markers+text',
    text=risk_return_df.index,
    textposition="top center",
    marker=dict(size=10)
))


So, AAPL shows the lowest risk combined with a positive average daily return, suggesting a more stable investment with consistent returns. GOOG has higher volatility than AAPL and, on average, a slightly negative daily return, indicating a riskier and less rewarding investment during this period.
---

MSFT shows moderate risk with the highest average daily return, suggesting a potentially more rewarding investment, although with higher volatility compared to AAPL. NFLX exhibits the highest risk and a negative average daily return, indicating it was the most volatile and least rewarding investment among these stocks over the analyzed period.
---

##Summary
###So, this is how you can perform a Quantitative Analysis of the Stock Market using Python. Quantitative Analysis in the stock market is a financial methodology that utilizes mathematical and statistical techniques to analyze stocks and financial markets..